File Structure:

Original Image: ./assets/images/sagittal_mouse.nii

Organs: ./data
- Contains .nii files for organ voxels, and .obj files for meshes

Annotations: ./annotations

Image Measurements: ./assets/images/nii-images/

Import Libraries

In [1]:
from os.path import join
from os import listdir
import os

import json
import numpy as np
from numpy.linalg import inv
from skimage.measure import marching_cubes
from scipy.ndimage import zoom
import cv2
import nibabel as nib
import meshio
import pymeshlab as ml
from pymeshlab.pmeshlab import Mesh as PMLMesh

import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.renderers.default = "notebook_connected"

Define global variables

In [2]:
WORKDIR = os.getcwd()
ANNOTATION_FOLDER = join(WORKDIR, "..", "annotations")
MOUSE_NIFTI_FILE = join(WORKDIR, "..", "assets", "images", "sagittal_mouse.nii")
MESHES_AND_VOXELS_FOLDER = join(WORKDIR, "..", "data")
IMAGE_FILES = join(WORKDIR, "..", "assets", "images", "nii-images")

Define the affine transformation matrix 

In [3]:
AFFINE = np.array(
    [[ 0.16952001,  0.,          0.,         21.58835983],
    [ 0.,          0.16952001,  0.,         21.69745827],
    [ 0.,          0.,          0.168538,   95.23029327],
    [ 0.,          0.,          0.,          1.        ]])

INV_AFFINE = inv(AFFINE)

Create mesh processor functions and library

In [4]:
class Mesh:
    def __init__(self, vertices:np.ndarray, faces:np.ndarray) -> None:
        """
        Parameters
        vertices: input vertices from marching_cubes
        faces: input faces from marching_cubes
        """
        self.vertices = vertices
        self.faces = faces
    
    def saveMesh(self, file_path:str) -> bool:
        """
        Parameters
        file_path: path to save the mesh to
        Returns
        true on success, false on failure
        """
        try:
            mesh = meshio.Mesh(self.vertices, {"triangle": self.faces})
            meshio.write(file_path, mesh, file_format="obj")

            return True
        except:
            return False
    
    def getVertices(self) -> np.ndarray:
        return self.vertices
    
    def getFaces(self) -> np.ndarray:
        return self.faces

def generate_mesh_from_voxels(voxels:np.ndarray, *, threshold:int=None, step_size:int=1, file_path:str=None) -> Mesh:
    """
    Parameters:
    voxels: 3D array of voxels (from nifti, or generated)
    threshold: threshold level to include from voxels in volume mesh. if not set, threshold is 95% of the maximum value
    step_size: step size for creating vertices
    file_path: file path to save .obj file to. if not set, no file is saved
    
    Returns:
    Mesh: mesh object of vertices and faces
    """
    if (threshold == None):
        maxval = np.max(voxels)
        threshold = int(maxval * 0.95)
    vertices, faces, _, _ = marching_cubes(voxels, level=threshold, step_size=step_size)
    mesh = Mesh(vertices, faces)
    if (file_path != None):
        mesh.saveMesh(file_path)

    return mesh

def read_mesh_from_file(file_path:str) -> Mesh:
    """
    Parameters:
    file_path: .obj file path to read mesh from

    Returns:
    Mesh: mesh object of vertices and faces
    """
    mesh = meshio.read(file_path)
    vertices = mesh.points
    faces = mesh.cells_dict["triangle"]

    return Mesh(vertices, faces)

def visualize_meshes(meshes:list) -> go.Figure:
    """
    Parameters:
    meshes: list of all Meshes you want to render

    Returns:
    figure: Plotly GO figure
    """
    data = []
    for mesh in meshes:
        vertices = mesh.getVertices()
        faces = mesh.getFaces()
        go_mesh = go.Mesh3d(
            x=vertices[:, 0], 
            y=vertices[:, 1], 
            z=vertices[:, 2], 
            i=faces[:, 0],
            j=faces[:, 1],
            k=faces[:, 2]
        )
        data.append(go_mesh)
    figure = go.Figure(data=data)
    return figure

def decimate_mesh(mesh: Mesh, num_vertices: int) -> Mesh:
    """
    Parameters:
    mesh: input Mesh object
    num_vertices: the target number of vertices to decimate to

    Returns:
    mesh: decimated mesh
    """
    num_faces = 3 * num_vertices
    ms = ml.MeshSet()
    pml_mesh = PMLMesh(mesh.vertices, mesh.faces)
    ms.add_mesh(pml_mesh)
    while (ms.current_mesh().vertex_number() > num_vertices):
        ms.apply_filter("meshing_decimation_quadric_edge_collapse", targetfacenum=num_faces, preservenormal=True)
        num_faces = num_faces - (ms.current_mesh().vertex_number() - num_vertices)
    
    m = ms.current_mesh()
    print('Output mesh has', m.vertex_number(), 'vertex and', m.face_number(), 'faces')
    return Mesh(m.vertex_matrix(), m.face_matrix())

Define Atlas and Organ objects

In [5]:
class Atlas:
    def __init__(self, annotationDirectory, calibrationAnnotation, save=False) -> None:
        # TODO: Get directory length
        self.calibration: float = None
        self.xOffset: int = None
        self.yOffset: int = None
        self.zOffset: int = None

        self.atlasDims: tuple = None
        self.affine = None

        self.organs = {}

        self.annotationDirectory = annotationDirectory

        ct = nib.load("./assets/images/sample/CT_TS_HEUHR_In111_free_M1039_0h_220721-selfcal.nii")
        ct = nib.load(join("assets", "images", "sagittal_mouse.nii"))

        annFiles = listdir(annotationDirectory)
        numSlices = 0
        for file in annFiles:
            annList = json.load(open(join(self.annotationDirectory, file)))
            numSlices += len(annList)

        self.calibrateDepth(calibrationAnnotation, numSlices)
        self.calibrateImgSize(ct)
        self.constructAtlasFromList(annFiles, numSlices)
        self.constructImgVoxels(save=save)

    def constructAtlasFromList(self, fileList, numSlices):
        if (self.calibration == None):
            print("Requires calibration")
            return
        # get number of slices
        for file in fileList:
            path = join(self.annotationDirectory, file)
            if (path == self.calibrationFile):
                print("Calibration file. Skipping...")
                continue
            print(f"Reading file {file}")
            annotationList = json.load(open(path))
            for annotation in annotationList:
                fname = annotation["documents"][0]["name"]
                index = int(fname.split('.')[0].replace("rat",""))
                try:
                    for entity in annotation["annotation"]["annotationGroups"][0]["annotationEntities"]:
                        name = entity["name"]
                        try:
                            organ = self.organs[name]
                        except(KeyError):
                            organ = Organ(name, 
                                            numSlices, 
                                            self.calibration, 
                                            self.atlasDims,
                                            self.affine)
                            self.organs[name] = organ
                        organ.appendOrganSlice(index, entity)
                except:
                    print(annotation)

    def calibrateDepth(self, calibrationAnnotation, numSlices):
        f = open(calibrationAnnotation)
        annotation = json.load(f)[0]
        body = annotation["annotation"]["annotationGroups"][0]["annotationEntities"][0]
        domain = []
        for point in body["annotationBlocks"][0]["annotations"][0]["segments"][0]:
            domain.append(point[1])
        minZ = min(domain)
        maxZ = max(domain)
        diff = maxZ - minZ
        self.calibration = 1.0 * float(diff) / float(numSlices)
        self.calibrationFile = calibrationAnnotation
        return self.calibration
    
    def calibrateImgSize(self, inputNifti):
        """
        Parameters:
            inputNifti: nibabel.nifti1.Nifti1Image
        """
        # TODO: use nifi image size to calibrate canvas size for voxelclouds
        shape = inputNifti.get_fdata().shape
        self.atlasDims = shape
        self.affine = inputNifti.affine
        print(self.affine, type(self.affine))
        pass

    def constructImgVoxels(self, save=False):
        for organName in self.organs:
            organ = None
            try:
                organ = self.organs[organName]
            except:
                continue
            if (organ != None):
                print(f"Constructing voxel map for {organName}")
                organ.constructVoxelMap(save)
                print("Done")

class Organ:
    def __init__(self, name, numSlices, depth, dims, affine) -> None:
        self.name = name
        self.numSlices = numSlices
        self.slices = [[]] * numSlices
        self.scale = 1.0
        self.depth = depth * self.scale
        self.affine = affine
        # offset: [offset_x, offset_y, offset_z]
        self.offset = {
            "x": 250, # 124, # OFFSET_X - 36 * 6, # 424.2
            "y": 173, # 45, # OFFSET_Y - 80 * 6, # 33
            "z": 385 # 50 # 65
        }
        imgSliceDims = (numSlices, dims[1], dims[2])
        self.imageSlices: np.ndarray = np.zeros(imgSliceDims, dtype=np.uint8)
        self.voxelCloud: np.ndarray = np.zeros(dims, dtype=np.uint8)
    
    def appendOrganSlice(self, index, entity):
        for polygon in entity["annotationBlocks"][0]["annotations"]:
            polyPts = np.array(polygon["segments"][0].copy()).astype(int)
            for i, pt in enumerate(polyPts):
                polyPts[i] = [(self.scale * pt[1]) - self.offset['y'], (self.scale * pt[0]) - self.offset['x']]
            self.slices[index].append(polyPts)
            cv2.fillPoly(self.imageSlices[index], pts=[polyPts], color=(255, 255, 255))

    def constructVoxelMap(self, save=False):
        for z in range(self.voxelCloud.shape[0]):
            # i: voxel layer
            i = z - self.offset['z']
            index = float(i) / self.depth
            ind0 = int(np.floor(index))
            if (ind0 < 0):
                continue
            if (ind0 + 1 >= len(self.imageSlices)):
                break
            img0 = self.imageSlices[ind0]
            img1 = self.imageSlices[ind0 + 1]
            img0 = cv2.GaussianBlur(img0, (27, 27), cv2.BORDER_DEFAULT)
            img1 = cv2.GaussianBlur(img1, (27, 27), cv2.BORDER_DEFAULT)
            alpha = (float(i % int(np.round(self.depth)) ) ) / (self.depth)
            additiveImage = np.add(img0 *  (1.0 - alpha), img1 * alpha)
            self.voxelCloud[z][np.where(additiveImage > 196)] = 255
        
        self.customCalibration()
        self.generateMesh(save)

        if (save):
            img = nib.Nifti1Image(self.voxelCloud, self.affine)
            nib.save(img, f"./data/{self.name}.nii")
            print(f"Saved {self.name} image at ./data/{self.name}.nii")
            del self.imageSlices
            del self.slices
            del self.voxelCloud
    
    def customCalibration(self):
        self.voxelCloud = np.swapaxes(self.voxelCloud, 0, 1);
        self.voxelCloud = self.voxelCloud[::-1,::-1,::]

        # smooth between slices
        for i, img in enumerate(self.voxelCloud):
            # smoothImg = 
            self.voxelCloud[i] = cv2.GaussianBlur(img, (27, 27), cv2.BORDER_DEFAULT)
        
    def generateMesh(self, save=False):
        threshold = 50
        step_size = 3
        print("Getting mesh")
        vertices, faces, _, _ = marching_cubes(self.voxelCloud, level=threshold, step_size=step_size)
        self.vertices = vertices
        self.faces = faces
        if (save):
            self.saveMesh()

    def getMesh(self):
        return self.vertices, self.faces

    def saveMesh(self):
        mesh = meshio.Mesh(self.vertices, {"triangle": self.faces})
        meshio.write(f"./data/{self.name}.obj", mesh, file_format="obj")

Generate atlas if it doesn't exist yet

In [7]:
data = listdir(MESHES_AND_VOXELS_FOLDER)
if (len(data) == 0):
    atlas = Atlas(join("assets", "annotations"), join("assets", "annotations", "calibrate.json"), save=True)

Write NIFTI image rescaling algorithm and define scale factor

In [6]:
def rescale_nifti(nifti_file, scale):
    image = nib.load(nifti_file)
    image_data = image.get_fdata()
    rescaled_data = zoom(image_data, scale, order=1)
    rescaled_image = nib.Nifti1Image(rescaled_data, AFFINE)
    del image
    del image_data
    return rescaled_image

SCALE_FACTOR = 0.3

Rescale the IMAIOS mouse image

In [12]:
rescaled_image = rescale_nifti(MOUSE_NIFTI_FILE, SCALE_FACTOR)
nib.save(rescaled_image, "../assets/images/scaled_mouse.nii")


For each organ in the atlas, rescale it to the same scale factor

In [8]:
for file in listdir(MESHES_AND_VOXELS_FOLDER):
    scaled = join(MESHES_AND_VOXELS_FOLDER, "scaled")
    os.makedirs(scaled, exist_ok=True)
    if (file.endswith(".nii")):
        rescaled_image = rescale_nifti(join(MESHES_AND_VOXELS_FOLDER, file), SCALE_FACTOR)
        nib.save(rescaled_image, join(scaled, f"scaled_{file}"))

View atlas and skeletal meshes

In [9]:
# Test liver decimation
liver = read_mesh_from_file(join(MESHES_AND_VOXELS_FOLDER, "Liver.obj"))
mesh = decimate_mesh(liver, 4000)
visualize_meshes([mesh]).show()

Output mesh has 4000 vertex and 8000 faces


In [11]:
meshes = []

for file in listdir(join(MESHES_AND_VOXELS_FOLDER, "scaled")):
    if (file.endswith(".nii")):
        print(file)
        voxels = nib.load(join(MESHES_AND_VOXELS_FOLDER, "scaled", file)).get_fdata()
        try:
            mesh = generate_mesh_from_voxels(voxels, threshold=45)
            mesh = decimate_mesh(mesh, 1500)
            meshes.append(mesh)
        except:
            print("Error")
        finally:
            del voxels
        print("Mesh created")
    
fig = visualize_meshes(meshes)
fig.show()

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'c:\\Users\\Andy\\Documents\\Repos\\gi-tract-atlas\\src\\..\\data\\scaled'

View the skeleton graph

In [111]:
# define node class
class SkNode:
    def __init__(self, points, _upstream):
        self.points = points
        self._target_upstream = _upstream
        self.target = None
        self.vec3d = np.zeros(3) # [x, y, z]
        self.affine = np.zeros((4, 4)) # [x, y, z, offset] affine transformation matrix

    
    def set_target(self, target):
        self.target = target
        self.compute_normalized_dir_vec(target)
    
    def compute_normalized_dir_vec(self, target):
        if (self.target == None):
            self.target = target
            # use target
        dir = (self.target.points - self.points)[0:3]
        mag = np.linalg.norm(dir)
        self.vec3d = dir / mag
        print(self.vec3d)
        return self.vec3d

    def calibrate_affine_matrix(self):
        pass

In [112]:
# Load skeleton graph
import csv

SKELETON_GRAPH_PATH = join(WORKDIR, "..", "atlas", "skeleton_graphs", "mouse_skeleton_graph.csv")

skeleton_graph = {}

xoffset = -48
yoffset = -303
zoffset = 300

with open(SKELETON_GRAPH_PATH) as csvfile:
    graph_data = csv.reader(csvfile, delimiter=",")
    for row in graph_data:
        name = row[0]
        try:
            world_coordinates = np.array([float(x) for x in row[1].split(",")]).astype(np.float32)
            world_coordinates = np.append(world_coordinates, 1)
            pixel_coordinates = np.dot(INV_AFFINE, world_coordinates)
            pixel_coordinates[0] = -pixel_coordinates[0] - xoffset
            pixel_coordinates[1] = pixel_coordinates[1] - yoffset
            pixel_coordinates[2] = -pixel_coordinates[2] - zoffset
        except Exception as e:
            print(e)
            continue
        _to = row[2]
        node = SkNode(pixel_coordinates, _to)
        skeleton_graph[name] = node
    
for name in skeleton_graph:
    node = skeleton_graph[name]
    for targ_node in skeleton_graph:
        if (targ_node in node._target_upstream):
            node.set_target(skeleton_graph[targ_node])
    
print(skeleton_graph)
print(INV_AFFINE)

could not convert string to float: 'Coordinates (x'
[ 0.          0.03518968 -0.99938065]
[ 0.         -0.01758471 -0.99984538]
[ 0.          0.59246037 -0.8055996 ]
[-0.03261938  0.14925838 -0.98826004]
[-0.02564562 -0.06335977 -0.99766119]
[ 0.00780201 -0.0922473  -0.99570556]
[ 0.81904474 -0.02301448  0.57326786]
[ 0.36193424  0.08914106 -0.92793183]
[-0.24745164 -0.72181541  0.64633598]
[-0.80322728  0.          0.59567268]
[-0.34580167  0.32743093 -0.87932371]
[ 0.22111299 -0.8507228   0.47684355]
[ 0.3376604  -0.13174861 -0.93200202]
[ 0.44491064 -0.88982128 -0.10135387]
[-0.21425798  0.10748254 -0.97084552]
[-0.29566813 -0.15571235 -0.94251473]
[-0.25632267 -0.94911082 -0.18299544]
[ 0.14780934  0.13610166 -0.97960642]
{'Centre of head': <__main__.SkNode object at 0x165ae25d0>, 'Atlantoaxial joint (joint head and neck)': <__main__.SkNode object at 0x165ae3990>, 'Cervical Vertebrae (neck bend vertebrae)': <__main__.SkNode object at 0x165ae2fd0>, 'Thoracic Vertebrae (back spine be

In [113]:
# generate lines arrays
xs = []
ys = []
zs = []

xoffset = -48
yoffset = -303
zoffset = 300

for name in skeleton_graph:
    node = skeleton_graph[name]
    target = node.target
    if (target == None):
        continue
    xs.append(node.points[0])
    ys.append(node.points[1])
    zs.append(node.points[2])
    
    xs.append(target.points[0])
    ys.append(target.points[1])
    zs.append(target.points[2])

    xs.append(None)
    ys.append(None)
    zs.append(None)

In [114]:
# get skeleton mesh
# skel_mesh = read_mesh_from_file("../data/ScaledSkeleton.obj")
# skel_fig = visualize_meshes([skel_mesh]).data[0]

In [115]:
import plotly.graph_objs as go
from plotly.offline import plot

def get_vector_go(origin, dir):
    # normalize direction
    if np.linalg.norm(dir) != 1:
        dir = dir / np.linalg.norm(dir)
    
    dir *= 40

    to_x = origin[0] + dir[0]
    to_y = origin[1] + dir[1]
    to_z = origin[2] + dir[2]

    vector = go.Scatter3d( 
        x = [origin[0], to_x],
        y = [origin[1], to_y],
        z = [origin[2], to_z],
        marker = dict( size = 1,
            color = "rgb(0, 0, 255)"
        ),
        line = dict(
            color = "rgb(0, 0, 255)",
            width = 2
        )
    )
    tip = go.Cone (
        x = [to_x],
        y = [to_y],
        z = [to_z],
        u = [0.5 * dir[0]],
        v = [0.5 * dir[1]],
        w = [0.5 * dir[2]],
        colorscale = [[0, "blue"], [1, "blue"]]
    )
    return vector, tip

# vec, cone = get_vector_go([0, 0, 0], [1, 1, 1])
# # data = [points,vector,tip]
# data = [vec, cone]
# layout = go.Layout(margin = dict( l = 0,
#                                   r = 0,
#                                   b = 0,
#                                   t = 0)
#                   )
# fig = go.Figure(data=data,layout=layout)
# fig.show()

In [118]:
# plot figure
skel_graph = go.Scatter3d(
        x=xs,
        y=ys,
        z=zs,
        # mode="lines",
        line_width=4,
        line_color="red"
    )

fig = make_subplots()

for name in skeleton_graph:
    node = skeleton_graph[name]
    vec, tip = get_vector_go(node.points[0:3], node.vec3d)
    fig.add_trace(vec)
    fig.add_trace(tip)
    

# fig.add_trace(skel_fig)
fig.add_trace(skel_graph)

layout = go.Layout(
    scene = dict(
        aspectmode="data", aspectratio=dict(x=1, y=1, z=1)
    )
)

fig.update_layout(layout)

fig.show()

/var/folders/vr/gvp8byjj5x36z1qnf8lytrxr0000gn/T/ipykernel_8359/2247655367.py:7: RuntimeWarning:

invalid value encountered in divide



Perform automatic calculation of mouse image for each organ

Data structure:
Image name:
- Organ name
    - Mean response
    - Calculated SUV

First, we should set the calibration factor

In [158]:
CALIBRATION_FACTOR = 441.23

Perform DFS through folder structure

In [17]:
def list_files(filepath, filetype):
    paths = []
    for root, dirs, files in os.walk(filepath):
        for file in files:
            if file.lower().endswith(filetype.lower()):
                paths.append(join(root, file))
    return paths

Get calibrated organ images

In [27]:
calibrated_dir = join(MESHES_AND_VOXELS_FOLDER, "calibrated")
organs = {}

for organ_file in listdir(calibrated_dir):
    organ_name = organ_file.split(".")[0]
    organs[organ_name] = nib.load(join(calibrated_dir, organ_file)).get_fdata()

In [29]:
spect_files = list_files(IMAGE_FILES, "ac.nii")
calculations = [["File name", "Stomach", "Liver", "Duodenum", "Jejunum", "Ileum", "Cecum", "Colon", "Rectum"]]

import csv

for nii_path in spect_files:
    img = nib.load(nii_path).get_fdata()
    row = [nii_path.split("\\")[-1].split("/")[-1], 0., 0., 0., 0., 0., 0., 0., 0.]
    for organ_name in organs:
        organ = organs[organ_name]
        contained_response = img[np.where(organ > 50)]
        mean = np.mean(contained_response)
        idx = calculations[0].index(organ_name)
        row[idx] = mean
    calculations.append(row)

UnsupportedOperation: not writable

In [35]:
with open(join(MESHES_AND_VOXELS_FOLDER, "measurements.csv"), "w") as f:
    writer = csv.writer(f)
    writer.writerows(calculations)

Create Skeleton Mesh

In [127]:
scaled_mouse = join(WORKDIR, "..", "assets", "images", "scaled_mouse.nii")
ms_nifti = nib.load(scaled_mouse).get_fdata()
mesh = generate_mesh_from_voxels(ms_nifti, threshold=200, step_size=3, file_path="../data/ScaledSkeleton.obj")